In [77]:
import nltk
from nltk.corpus import wordnet as wn

In [48]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn

# Define una función para calcular la polaridad de un synset
def synset_polarity(synset):
    senti_synset = swn.senti_synset(synset.name())
    if not senti_synset:
        # Si el synset no tiene entrada en SentiWordNet, devuelve polaridad neutra
        return 0
    else:
        # Calcula la polaridad positiva y negativa del synset
        pos_score = senti_synset.pos_score()
        neg_score = senti_synset.neg_score()
        # Calcula la polaridad total normalizada
        polarity = (pos_score - neg_score) / (pos_score + neg_score + 0.001)
        return polarity

# Define una función para obtener los synsets y polaridades de cada adjetivo en un conjunto de texto
def get_polarities(text, tags = ['JJ', 'JJR', 'JJS']):
    words = nltk.word_tokenize(text)
    important_words = []

    for word, pos in nltk.pos_tag(words):
        if pos in tags: 
            important_words.append((word, pos, wn.synsets(word)))

    polarities = []

    for word, pos, synsets in important_words:
        polarity = 0
        count = 0

        for synset in synsets:
            # Calcula la polaridad de cada synset y la suma
            polarity += synset_polarity(synset)
            count += 1

        if count > 0:
            # Calcula la polaridad media
            polarity /= count
            
        # Añade la polaridad media a la lista de polaridades
        polarities.append(polarity)
    return polarities




In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Carga los datos
df = pd.read_csv('reviews.csv')
X = df.drop('sentiment', axis=1)
y = df['sentiment']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

In [49]:
adj = ['JJ', 'JJR', 'JJS']
noun = ['NN', 'NNS']
verb = ['VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ']
adv = ['RB', 'RBR', 'RBS']

noun_adj_adv = noun + adj + adv
noun_adj_adv_verb = noun + adj + adv + verb

FN = 0 # False Negative
FP = 0 # False Positive
TP = 0 # True Positive
TN = 0 # True Negative

# Itera sobre los datos de test
for text, pred in zip(X_test['review'], y_test):
    polarities = get_polarities(text, noun_adj_adv)     
    output = 0

    # Si la suma de las polaridades es positiva, la predicción es positiva
    if sum(polarities) > 0:
        output = 1
    
    
    if output == 1 and pred:
        TP += 1
    elif output == 1 and not pred:
        FP += 1
    elif output == 0 and pred:
        FN += 1
    elif output == 0 and not pred:
        TN += 1

accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP)
recall = TP / (TP + FN)
f1 = 2 * (precision * recall) / (precision + recall)

print("Accuracy: ", accuracy)
print("Precision: ", precision)
print("Recall: ", recall)
print("F1: ", f1)

    
        
    

Accuracy:  0.59
Precision:  0.5623100303951368
Recall:  0.9024390243902439
F1:  0.6928838951310862


In [76]:
text = "I hate the movie. It was incandcent"
tags = ['JJ']

words = nltk.word_tokenize(text)
important_words = []
for word, pos in nltk.pos_tag(words):
    if pos in adj:
        print(word, pos)


incandcent JJ
